In [1]:
from Acesso import Login
from Query import  Query
import pandas as pd

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
querys={
    
    'Vendas':
    
    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME,@DIAS SMALLINT

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS date),101)

    SET @DIAS=DAY(@DTBASE)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=DATEADD(MONTH,-1,DATEADD(DAY,1,DATEADD(DAY,@DIAS*-1,@DTFIM)))

    SELECT * FROM netfeira.vw_venda_estatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Tipo de Operação]='VENDAS'
    ORDER BY [Data de Faturamento]  
    
    """,
    
    'Meta':
    
    """
    
    SELECT * FROM netfeira.vw_metas
    WHERE [Meta R$]>0
    
    """,
    
    'Vendedor':
    
    """

    SELECT v.[ID Vendedor],v.[Nome Resumido],s.Equipe,s.[ID Sup],s.Supervisor,s.[DDD Sup],s.[Telefone Sup],
    s.[ID Gerente],s.Gerente,s.[DDD Gerente],s.[Telefone Gerente]
    FROM netfeira.vw_vendedor v
    INNER JOIN netfeira.vw_supervisor s ON v.[ID Equipe]=s.[ID Equipe]
    WHERE v.[Status do Vendedor]='ATIVO'

    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """,
    
    'Calendario':
    
    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS date),101)

    SELECT COUNT(Data) AS [Útil],COUNT([Data Trabalhada])-1 AS [Trabalhado],COUNT(Data)-(COUNT([Data Trabalhada])-1) AS [Restante]
    FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(@DTBASE) AND MONTH(Data)=MONTH(@DTBASE) AND [Dia Útil]=1    

    """
    
}

In [4]:
s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [5]:
df=sql.CriarTabela(kwargs=querys)

In [6]:
df['Vendas'].columns

Index(['ID Empresa', 'Pedido', 'NFe', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'Origem', 'Situação', 'Data de Emissão',
       'Data de Faturamento', 'ID Cliente', 'ID Vendedor', 'SKU', 'Seq',
       'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor de VDA', 'Total Geral',
       'Total Venda', 'Comissão R$', 'COFINS R$', 'PIS R$', 'ICMS R$',
       'ICMS ST R$', 'Peso Bruto KG', 'Peso Líquido KG', 'Margem Bruta R$'],
      dtype='object')

In [7]:
df['Vendedor'].columns

Index(['ID Vendedor', 'Nome Resumido', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

In [8]:
df['Meta'].columns

Index(['ID Vendedor', 'Meta R$'], dtype='object')

In [9]:
df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'Rede', 'CNPJ Base', 'CNPJ Caracter',
       'Razão Social', 'Nome Fantasia', 'Tipo de Cliente', 'Status do Cliente',
       'ID Segmento', 'Matriz', 'Crédito', 'Data de Cadastro',
       'Primeira Compra', 'Última Compra', 'Dias Compra', 'Tabela',
       'Condição de Pagto', 'Prazo Pagto', 'Pagamento', 'CEP', 'Endereço',
       'Bairro', 'Município', 'Numero', 'Complemento', 'Região', 'UF', 'DDD',
       'Contato', 'Limite de Crédito', 'Principal', 'E-mail Cliente',
       'Tributação do Cliente', 'ID Rota', 'Latitude', 'Longitude',
       'Desc Financeiro'],
      dtype='object')

In [10]:
df['Calendario'].columns

Index(['Útil', 'Trabalhado', 'Restante'], dtype='object')

In [11]:
from datetime import datetime

In [81]:
dt_atu=datetime.now().date()

dia_util=df['Calendario']['Útil'].values[-1]

dia_trab=df['Calendario']['Trabalhado'].values[-1]

dia_restante=df['Calendario']['Restante'].values[-1]

In [110]:
#criar base de dados

df['Temp']=df['Vendas'].loc[(df['Vendas']['Data de Faturamento'].dt.year==dt_atu.year)&(df['Vendas']['Data de Faturamento'].dt.month==dt_atu.month)]

df['Temp']=df['Temp'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Temp']=df['Temp'].merge(df['Meta'],on='ID Vendedor',how='inner')

df['Temp']=df['Temp'].merge(df['Vendedor'],on='ID Vendedor',how='inner')

df['Temp']=df['Temp'].groupby(['Equipe'],as_index=False).agg({'Total Venda':'sum','Meta R$':'sum'})

df['Temp']['Projeção']=round((df['Temp']['Total Venda']/dia_trab)*dia_util,2)

df['Temp']['Status']=df['Temp'].apply(lambda info: 1 if info['Projeção']>=info['Meta R$'] else 0,axis=1)

df['Temp']=df['Temp'].loc[df['Temp']['Status']==0]

equipes=df['Temp']['Equipe'].tolist()

codigos=df['Vendedor']['ID Vendedor'].loc[df['Vendedor']['Equipe'].isin(equipes)].tolist()

In [111]:
#consolidado dados

df['Temp']=df['Vendas'].loc[(df['Vendas']['Data de Faturamento'].dt.year==dt_atu.year)&(df['Vendas']['Data de Faturamento'].dt.month==dt_atu.month)&(df['Vendas']['ID Vendedor'].isin(codigos))]

df['Temp']=df['Temp'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Temp']=df['Temp'].merge(df['Meta'],on='ID Vendedor',how='inner')

df['Temp']=df['Temp'].merge(df['Vendedor'],on='ID Vendedor',how='inner')[['ID Vendedor', 'Nome Resumido', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente','Total Venda','Meta R$']]

df['Temp']['Projeção']=round((df['Temp']['Total Venda']/dia_trab)*dia_util,2)

df['Temp']['Status']=df['Temp'].apply(lambda info: 1 if info['Projeção']>=info['Meta R$'] else 0,axis=1)

df['Temp']=df['Temp'].loc[df['Temp']['Status']==0]

df['Temp'].sort_values('Total Venda',ascending=False,inplace=True)

codigos=df['Temp']['ID Vendedor'].tolist()

In [112]:
valores=[]

diferenca=[]

for c in codigos:
    
    df['Diario']=df['Vendas'].loc[(df['Vendas']['Data de Faturamento'].dt.year==dt_atu.year)&(df['Vendas']['Data de Faturamento'].dt.month==dt_atu.month)&(df['Vendas']['ID Vendedor']==c)]
    
    meta=df['Temp']['Meta R$'].loc[df['Temp']['ID Vendedor']==c].values[-1]
    
    meta=round(meta/dia_util,2)
    
    df['Diario']=df['Diario'].groupby(['Data de Faturamento'],as_index=False).agg({'Total Venda':'sum'})
    
    df['Diario'].sort_values('Data de Faturamento',ascending=True,inplace=True)
    
    df['Diario']['Meta']=meta
    
    df['Diario']['Dif']=round(df['Diario']['Total Venda']-df['Diario']['Meta'],2)
        
    dif=df['Diario']['Dif'].sum()
    
    total=round(abs(dif)+meta if dif<0 else meta-dif,2)
    
    diferenca.append(dif)
    
    valores.append(total)
    
    pass

df['Temp']['Meta Diária']=round(df['Temp']['Meta R$']/dia_util,2)

df['Temp']['Débito Diário']=diferenca

df['Temp']['Á Realizar']=valores

df['Temp']['Á Realizar']=round(df['Temp']['Á Realizar']/dia_restante,2)

df['Temp'].sort_values('Débito Diário',ascending=True,inplace=True,ignore_index=True)


In [114]:
df['Temp'].loc[df['Temp']['Débito Diário']>0,'Meta Diária']=df['Temp'].loc[df['Temp']['Débito Diário']>0].apply(lambda info: round((info['Meta R$']-info['Total Venda'])/dia_restante,2),axis=1)

In [116]:
df['Temp'].loc[df['Temp']['Débito Diário']>0,'Á Realizar']=df['Temp'].loc[df['Temp']['Débito Diário']>0,'Meta Diária']

In [121]:
codigos=df['Temp']['ID Vendedor'].tolist()

In [123]:
df['Vendas']['Mês']=df['Vendas']['Data de Faturamento'].dt.month

In [151]:
#relação de clientes

df['Lista']=df['Vendas'].loc[df['Vendas']['ID Vendedor'].isin(codigos)].groupby(['ID Cliente','ID Vendedor','Mês'],as_index=False).agg({'Total Venda':'sum'})

df['Lista']=df['Lista'].pivot(index=['ID Cliente','ID Vendedor'],columns='Mês',values='Total Venda').reset_index()

colunas=[l for l in df['Lista'].columns if str(l).isnumeric()]

meses={1:'JAN',2:'FEV',3:'MAR',4:'ABR',5:'MAI',6:'JUN',7:'JUL',8:'AGOS',9:'SET',10:'OUT',11:'NOV',12:'DEZ'}

for i,c in enumerate(colunas):
    
    if(i==0):
        
        df['Lista']=df['Lista'].loc[df['Lista'][c]>0]
        
        pass
    
    df['Lista'].loc[df['Lista'][c].isnull(),c]=0
    
    df['Lista'].rename(columns={c:meses[c]},inplace=True)
    
    pass

df['Lista'].loc[df['Lista'][df['Lista'].columns[-1]]<=0,'Positivado']='N'

df['Lista'].loc[df['Lista']['Positivado'].isnull(),'Positivado']='S'

df['Lista']=df['Cliente'].merge(df['Lista'],on='ID Cliente',how='inner')

df['Lista']=df['Vendedor'].merge(df['Lista'],on='ID Vendedor',how='inner')

df['Lista']


,ID Vendedor,Nome Resumido,Equipe,ID Sup,Supervisor,DDD Sup,Telefone Sup,ID Gerente,Gerente,DDD Gerente,Telefone Gerente,ID Cliente,CNPJ,Rede,CNPJ Base,CNPJ Caracter,Razão Social,Nome Fantasia,Tipo de Cliente,Status do Cliente,ID Segmento,Matriz,Crédito,Data de Cadastro,Primeira Compra,Última Compra,Dias Compra,Tabela,Condição de Pagto,Prazo Pagto,Pagamento,CEP,Endereço,Bairro,Município,Numero,Complemento,Região,UF,DDD,Contato,Limite de Crédito,Principal,E-mail Cliente,Tributação do Cliente,ID Rota,Latitude,Longitude,Desc Financeiro,NOV,DEZ,Positivado
0,ALEJESUS,ALEXANDRE REPRESENTACOES,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,27871,45141030000156,AVULSO,45141030,14,DAJE ROMA LTDA,DAJE ROMA,J,ATIVO,0004,None,NORMAL,2022-05-19,NaT,2022-12-07,20,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,5415050.0,"RUA MATEUS GROU, 19",PINHEIROS,SÃO PAULO,19,,PINHEIROS,SP,11,21738000.0,800.00,ALEJESUS,PARALEGAL@GGP.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,PINH.,-23.565620,-46.683510,NaN,663.31,181.44,S
1,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5291,53345252000119,AVULSO,53345252,14,NELSON KIOSHI NAKADA-COML AGRICOLA LT,SACOLAO SAUDE SAO BE,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-08-13,2008-08-13,2022-12-19,8,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,9760030.0,RUA TAPAJOS,VILA SCOPEL,SÃO BERNARDO DO CAMPO,80,None,S.BERNARDO DO CAMPO,SP,11,41211211.0,5399.79,DINHO,RECEBIMENTO.SB@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ABC,-23.699263,-46.549910,NaN,3279.95,320.20,S
2,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5355,02846188000119,AVULSO,02846188,14,COMERCIO DE ALIMENTOS SANTA CRUZ LTDA,SACOLAO SAUDE STA CR,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-12-27,0,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,4121002.0,RUA SANTA CRUZ,VILA MARIANA,SÃO PAULO,1851,None,SAUDE,SP,11,50692999.0,3000.00,DINHO,NFE-SACOLAODASANTA@BLUESOFT.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,SAUDE,-23.602513,-46.618540,NaN,11655.26,13134.28,S
3,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5356,05327676000108,AVULSO,05327676,14,SACOLAO SANTA CECILIA LTDA,SACOLAO SAUDE SANTA,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-11-30,27,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,1226010.0,RUA DAS PALMEIRAS,VILA BUARQUE,SÃO PAULO,278,None,VILA BUARQUE,SP,11,36663132.0,900.00,DINHO,RECEBIMENTO.SC@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,CENTRO,-23.536427,-46.652443,NaN,2921.95,-569.05,N
4,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5357,60272747000120,AVULSO,60272747,14,KYO COMERCIAL AGRICOLA LTDA,SACOLAO SAUDE STO AN,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-12-16,11,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,9210580.0,AVENIDA DOS ESTADOS,PARQUE CENTRAL,SANTO ANDRÉ,2195,None,SANTO ANDRÉ,SP,11,49968888.0,8534.57,DINHO,RECEBIMENTO.SA@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ABC,-23.632460,-46.537459,NaN,6512.81,348.28,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,VIVIANCR,VIVIAN FIGUEIR,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,25283,41150000000155,AVULSO,41150000,14,MERCADO IRMAOS PEREIRA LIMA LTDA,MERCADO IRMAOS,J,ATIVO,0001,None,NORMAL,2021-03-25,NaT,2022-11-28,29,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,7131240.0,"RUA JOSEFINA DALCIN CASEIRO, 450",JARDIM DA MAMÃE,GUARULHOS,450,,GUARULHOS,SP,11,22796230.0,350.00,VIVIANCR,,PESSOA JURIDICA - CONTRI NORMAL,GUA,-23.434747,-46.522780,NaN,539.28,0.00,N
743,VIVIANCR,VIVIAN FIGUEIR,EQ

In [176]:
colunas={'ID Sup':'ID Gerente','ID Gerente':'ID Gerente'}

nomes={'ID Sup':'Supervisor','ID Gerente':'Gerente'}

tel_ddd={'ID Sup':'DDD Sup','ID Gerente':'DDD Gerente'}

tel_num={'ID Sup':'Telefone Sup','ID Gerente':'Telefone Gerente'}

for col1,col2 in colunas.items():
    
    codigos=df['Temp'][col1].unique().tolist()
    
    for c in codigos:
        
        cod=df['Temp'][col2].loc[df['Temp'][col1]==c].unique().tolist()[-1]
        
        if(c==cod and col1=='ID Sup'):
            
            continue
                
        display(df['Lista'].loc[df['Lista'][col1]==c])
        
        pass
    
    pass


,ID Vendedor,Nome Resumido,Equipe,ID Sup,Supervisor,DDD Sup,Telefone Sup,ID Gerente,Gerente,DDD Gerente,Telefone Gerente,ID Cliente,CNPJ,Rede,CNPJ Base,CNPJ Caracter,Razão Social,Nome Fantasia,Tipo de Cliente,Status do Cliente,ID Segmento,Matriz,Crédito,Data de Cadastro,Primeira Compra,Última Compra,Dias Compra,Tabela,Condição de Pagto,Prazo Pagto,Pagamento,CEP,Endereço,Bairro,Município,Numero,Complemento,Região,UF,DDD,Contato,Limite de Crédito,Principal,E-mail Cliente,Tributação do Cliente,ID Rota,Latitude,Longitude,Desc Financeiro,NOV,DEZ,Positivado
117,JUVILARE,JULIANA SANTOS,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,6589,66603671000155,AVULSO,66603671,14,CASA TARTIN COMERCIO DE ALIMENTOS LTDA,CASA TARTIN,J,ATIVO,0014,None,NORMAL,2009-08-20,2009-08-20,2022-12-16,11,TABELA DE PREÇO,BOLETO BANCARIO 07 DIAS,7.0,BANCO,4205001.0,"RUA CIPRIANO BARATA, 1905",IPIRANGA,SÃO PAULO,1905,,IPIRANGA,SP,11,29175023.0,235.12,JUVILARE,CONTABILMELLO@YAHOO.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,SAUDE,-23.591999,-46.569460,NaN,125.47,320.04,S
118,JUVILARE,JULIANA SANTOS,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,18589,11082874000232,REDE,11082874,14,AMERICANA FRANQUIA LTDA EPP,AMERICANA FRANQUIA,J,ATIVO,0004,FRIDAYS,NORMAL,2017-01-13,2017-01-13,2022-12-21,6,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,4707900.0,"AVENIDA ROQUE PETRONI JÚNIOR 1089, 1089 - LOJA...",JARDIM DAS ACÁCIAS,SÃO PAULO,1089,LOJA LUC N 30-D/E-L,SANTO AMARO,SP,11,26140908.0,800.00,JUVILARE,NFE.XML@BRAZILAIRR.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ST.AMARO,-23.581404,-46.684628,NaN,4795.43,7910.28,S
119,JUVILARE,JULIANA SANTOS,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,19237,12809187000158,AVULSO,12809187,14,ESCOLA EDUCACAO INFANTIL KINDERGARTEN LT,ESCOLA EDUCACAO INFA,J,ATIVO,0019,None,NORMAL,2017-05-30,2017-05-30,2022-11-21,36,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,3322002.0,RUA AZEVEDO SOARES,VILA GOMES CARDIM,SÃO PAULO,2006,None,ANALIA FRANCO,SP,11,20942600.0,1000.00,JUVILARE,,PESSOA JURIDICA - ISENTA,ANALIA,-23.546510,-46.552430,NaN,1056.18,0.00,N
120,JUVILARE,JULIANA SANTOS,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,19826,20205435000180,AVULSO,20205435,14,SAO PAULO AIRPORT RESTAURANTES LIMITADA,WILLBOND,J,ATIVO,0010,FRIDAYS,NORMAL,2017-12-18,NaT,2022-12-27,0,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,7190100.0,"RODOVIA HÉLIO SMIDT, SN",AEROPORTO,GUARULHOS,SN,,GUARULHOS,SP,11,24457248.0,4500.00,JUVILARE,NFE.XML@BRAZILAIRR.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,GUA,-23.459952,-46.491079,NaN,8055.84,8940.70,S
121,JUVILARE,JULIANA SANTOS,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,21597,11082874000313,REDE,11082874,14,AMERICANA FRANQUIA S.A,AMERICANAS FRANQUIA,J,ATIVO,0018,FRIDAYS,NORMAL,2018-11-27,NaT,2022-12-20,7,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,2089900.0,"TRAVESSA CASALBUONO 120, 120 - SUC 025",VILA GUILHERME,SÃO PAULO,120,SUC 025,SANTANA,SP,11,997389030.0,1277.74,JUVILARE,NFE.XML@BRAZILAIRR.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,Z.NORTE,-23.517094,-46.616673,NaN,5447.91,7211.57,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,TALITA,TALITA OLIVEIRA,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,28953,29843688000153,AVULSO,29843688,14,A 2 I INDUSTRIA E COMERCIO DE ALIMENTOS LTDA,A2I INDUSTRIA E COME,J,ATIVO,0042,None,NORMAL,2022-11-10,NaT,2022-12-16,11,TABELA DE PREÇO,BOLETO BANCARIO 07 DIAS,7.0,BANCO,6701211.0,"RUA DAS GABIROBAS, 280",RESIDENCIAL RECANTO VERDE,COTIA,280,,COTIA,SP,11,945698150.0,400.00,TALITA,CONTATO@A2ISACHES.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,BUTA

,ID Vendedor,Nome Resumido,Equipe,ID Sup,Supervisor,DDD Sup,Telefone Sup,ID Gerente,Gerente,DDD Gerente,Telefone Gerente,ID Cliente,CNPJ,Rede,CNPJ Base,CNPJ Caracter,Razão Social,Nome Fantasia,Tipo de Cliente,Status do Cliente,ID Segmento,Matriz,Crédito,Data de Cadastro,Primeira Compra,Última Compra,Dias Compra,Tabela,Condição de Pagto,Prazo Pagto,Pagamento,CEP,Endereço,Bairro,Município,Numero,Complemento,Região,UF,DDD,Contato,Limite de Crédito,Principal,E-mail Cliente,Tributação do Cliente,ID Rota,Latitude,Longitude,Desc Financeiro,NOV,DEZ,Positivado
73,FABIANO,FABIANO COMERCIA,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,2349,66533696000120,REDE,66533696,14,HM SUPERMERCADOS LTDA,EMPORIUM SAO PAULO J,J,ATIVO,0001,EMPORIUM,NORMAL,2005-07-12,2005-07-12,2022-12-16,11,TABELA DE PREÇO,BOLETO BANCARIO 30 DIAS,30.0,BANCO,4079001.0,AVENIDA JUREMA,INDIANÓPOLIS,SÃO PAULO,271,None,MOEMA,SP,11,5.054800e+07,4495.70,RODRIGOA,NFE-JUR@EMPORIUMSAOPAULO.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ITAIM B,-23.609295,-46.658463,NaN,4351.34,2207.64,S
74,FABIANO,FABIANO COMERCIA,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,6586,02299215000180,AVULSO,02299215,14,HM HM SUPERMERCADOS LTDA,EMPORIUM SAO PAULO A,J,ATIVO,0001,EMPORIUM,NORMAL,2009-08-19,2009-08-19,2022-12-22,5,TABELA DE PREÇO,BOLETO BANCARIO 30 DIAS,30.0,BANCO,4511011.0,RUA AFONSO BRAS,VILA NOVA CONCEIÇÃO,SÃO PAULO,431,None,ITAIM BIBI,SP,11,3.848370e+07,6000.00,FABIANO,NFE-AFO@EMPORIUMSAOPAULO.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ITAIM B,-23.595346,-46.669887,NaN,5877.04,5113.56,S
75,FABIANO,FABIANO COMERCIA,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,7991,66533696000391,REDE,66533696,14,HM SUPERMERCADOS LTDA,EMPORIUM SAO PAULO I,J,ATIVO,0001,EMPORIUM,NORMAL,2010-06-16,2010-06-16,2022-12-20,7,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,4082002.0,AVENIDA IRAI,INDIANÓPOLIS,SÃO PAULO,675,None,MOEMA,SP,11,5.534460e+07,1710.41,RODRIGOA,NFE-IRA@EMPORIUMSAOPAULO.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ITAIM B,-23.612541,-46.660984,NaN,844.04,-169.84,N
76,FABIANO,FABIANO COMERCIA,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,9836,66533696000553,REDE,66533696,14,HM SUPERMERCADOS LTDA,EMPORIUM SAO PAULO -,J,ATIVO,0001,EMPORIUM,NORMAL,2011-12-13,2011-12-13,2022-12-23,4,TABELA DE PREÇO,BOLETO BANCARIO 30 DIAS,30.0,BANCO,4089010.0,ALAMEDA MARACATINS,INDIANÓPOLIS,SÃO PAULO,217,None,MOEMA,SP,0,1.111111e+10,204.16,RODRIGOA,NFE-MAR@EMPORIUMSAOPAULO.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ITAIM B,-23.603926,-46.657829,NaN,2336.98,0.00,N
77,FABIANO,FABIANO COMERCIA,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,15855,46852828000179,AVULSO,46852828,14,MERCADINHO SONGS LTDA EPP,SONG'S PLUS,J,ATIVO,0037,None,NORMAL,2016-01-26,2016-01-26,2022-12-22,5,TABELA DE PREÇO,BOLETO BANCARIO 28 DIAS,28.0,BANCO,1542000.0,"RUA DOM DUARTE LEOPOLDO, 184",CAMBUCI,SÃO PAULO,184,,LIBERDADE,SP,11,3.275077e+07,1107.34,FABIANO,MACIEL.CATIA@IG.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,CONSOLA,-23.568028,-46.623371,NaN,1426.48,2079.08,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,VIVIANCR,VIVIAN FIGUEIR,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,25283,41150000000155,AVULSO,41150000,14,MERCADO IRMAOS PEREIRA LIMA LTDA,MERCADO IRMAOS,J,ATIVO,0001,None,NORMAL,2021-03-25,NaT,2022-11-28,29,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,7131240.0,"RUA JOSEFINA DALCIN CASEIRO, 450",JARDIM DA MAMÃE,GUARULHOS,450,,GUARULHOS,SP,11,2.279623e+07,350.00,VIVIANCR,,PESSOA JURIDICA - CONTRI NORMAL,GUA,-23.434747,-46.522780,NaN,539.28,0.00,N
743,VIVIANCR,VIVIAN FIGUEIR,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGER

,ID Vendedor,Nome Resumido,Equipe,ID Sup,Supervisor,DDD Sup,Telefone Sup,ID Gerente,Gerente,DDD Gerente,Telefone Gerente,ID Cliente,CNPJ,Rede,CNPJ Base,CNPJ Caracter,Razão Social,Nome Fantasia,Tipo de Cliente,Status do Cliente,ID Segmento,Matriz,Crédito,Data de Cadastro,Primeira Compra,Última Compra,Dias Compra,Tabela,Condição de Pagto,Prazo Pagto,Pagamento,CEP,Endereço,Bairro,Município,Numero,Complemento,Região,UF,DDD,Contato,Limite de Crédito,Principal,E-mail Cliente,Tributação do Cliente,ID Rota,Latitude,Longitude,Desc Financeiro,NOV,DEZ,Positivado
0,ALEJESUS,ALEXANDRE REPRESENTACOES,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,27871,45141030000156,AVULSO,45141030,14,DAJE ROMA LTDA,DAJE ROMA,J,ATIVO,0004,None,NORMAL,2022-05-19,NaT,2022-12-07,20,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,5415050.0,"RUA MATEUS GROU, 19",PINHEIROS,SÃO PAULO,19,,PINHEIROS,SP,11,21738000.0,800.00,ALEJESUS,PARALEGAL@GGP.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,PINH.,-23.565620,-46.683510,NaN,663.31,181.44,S
1,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5291,53345252000119,AVULSO,53345252,14,NELSON KIOSHI NAKADA-COML AGRICOLA LT,SACOLAO SAUDE SAO BE,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-08-13,2008-08-13,2022-12-19,8,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,9760030.0,RUA TAPAJOS,VILA SCOPEL,SÃO BERNARDO DO CAMPO,80,None,S.BERNARDO DO CAMPO,SP,11,41211211.0,5399.79,DINHO,RECEBIMENTO.SB@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ABC,-23.699263,-46.549910,NaN,3279.95,320.20,S
2,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5355,02846188000119,AVULSO,02846188,14,COMERCIO DE ALIMENTOS SANTA CRUZ LTDA,SACOLAO SAUDE STA CR,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-12-27,0,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,4121002.0,RUA SANTA CRUZ,VILA MARIANA,SÃO PAULO,1851,None,SAUDE,SP,11,50692999.0,3000.00,DINHO,NFE-SACOLAODASANTA@BLUESOFT.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,SAUDE,-23.602513,-46.618540,NaN,11655.26,13134.28,S
3,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5356,05327676000108,AVULSO,05327676,14,SACOLAO SANTA CECILIA LTDA,SACOLAO SAUDE SANTA,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-11-30,27,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,1226010.0,RUA DAS PALMEIRAS,VILA BUARQUE,SÃO PAULO,278,None,VILA BUARQUE,SP,11,36663132.0,900.00,DINHO,RECEBIMENTO.SC@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,CENTRO,-23.536427,-46.652443,NaN,2921.95,-569.05,N
4,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5357,60272747000120,AVULSO,60272747,14,KYO COMERCIAL AGRICOLA LTDA,SACOLAO SAUDE STO AN,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-12-16,11,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,9210580.0,AVENIDA DOS ESTADOS,PARQUE CENTRAL,SANTO ANDRÉ,2195,None,SANTO ANDRÉ,SP,11,49968888.0,8534.57,DINHO,RECEBIMENTO.SA@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ABC,-23.632460,-46.537459,NaN,6512.81,348.28,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,ROBSONAN,ROBSON ANDRADE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,26469,45827425003718,REDE,45827425,14,SUPERMERCADOS IRMÃOS LOPES SA,LOPES SUPERMERCADOS,J,ATIVO,0001,LOPES SM,NORMAL,2021-11-01,NaT,2022-12-21,6,LOPES,DEPOSITO BANCARIO 35 DIAS,35.0,DEP CONTA,9860122.0,AVENIDA ROBERT KENNEDY - DE 1322 A 2300 - LADO...,INDEPENDÊNCIA,SÃO BERNARDO DO CAMPO,1538,GALPAO19 E N 1504 - GP 20,S.BERNARDO DO CAMPO,SP,11,111111111.0,290.15,ROBSONAN,LOPES.NFE@SUPERLOPES.COM.BR,PESSOA JURIDIC

,ID Vendedor,Nome Resumido,Equipe,ID Sup,Supervisor,DDD Sup,Telefone Sup,ID Gerente,Gerente,DDD Gerente,Telefone Gerente,ID Cliente,CNPJ,Rede,CNPJ Base,CNPJ Caracter,Razão Social,Nome Fantasia,Tipo de Cliente,Status do Cliente,ID Segmento,Matriz,Crédito,Data de Cadastro,Primeira Compra,Última Compra,Dias Compra,Tabela,Condição de Pagto,Prazo Pagto,Pagamento,CEP,Endereço,Bairro,Município,Numero,Complemento,Região,UF,DDD,Contato,Limite de Crédito,Principal,E-mail Cliente,Tributação do Cliente,ID Rota,Latitude,Longitude,Desc Financeiro,NOV,DEZ,Positivado
0,ALEJESUS,ALEXANDRE REPRESENTACOES,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,27871,45141030000156,AVULSO,45141030,14,DAJE ROMA LTDA,DAJE ROMA,J,ATIVO,0004,None,NORMAL,2022-05-19,NaT,2022-12-07,20,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,5415050.0,"RUA MATEUS GROU, 19",PINHEIROS,SÃO PAULO,19,,PINHEIROS,SP,11,21738000.0,800.00,ALEJESUS,PARALEGAL@GGP.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,PINH.,-23.565620,-46.683510,NaN,663.31,181.44,S
1,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5291,53345252000119,AVULSO,53345252,14,NELSON KIOSHI NAKADA-COML AGRICOLA LT,SACOLAO SAUDE SAO BE,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-08-13,2008-08-13,2022-12-19,8,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,9760030.0,RUA TAPAJOS,VILA SCOPEL,SÃO BERNARDO DO CAMPO,80,None,S.BERNARDO DO CAMPO,SP,11,41211211.0,5399.79,DINHO,RECEBIMENTO.SB@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ABC,-23.699263,-46.549910,NaN,3279.95,320.20,S
2,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5355,02846188000119,AVULSO,02846188,14,COMERCIO DE ALIMENTOS SANTA CRUZ LTDA,SACOLAO SAUDE STA CR,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-12-27,0,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,4121002.0,RUA SANTA CRUZ,VILA MARIANA,SÃO PAULO,1851,None,SAUDE,SP,11,50692999.0,3000.00,DINHO,NFE-SACOLAODASANTA@BLUESOFT.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,SAUDE,-23.602513,-46.618540,NaN,11655.26,13134.28,S
3,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5356,05327676000108,AVULSO,05327676,14,SACOLAO SANTA CECILIA LTDA,SACOLAO SAUDE SANTA,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-11-30,27,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,1226010.0,RUA DAS PALMEIRAS,VILA BUARQUE,SÃO PAULO,278,None,VILA BUARQUE,SP,11,36663132.0,900.00,DINHO,RECEBIMENTO.SC@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,CENTRO,-23.536427,-46.652443,NaN,2921.95,-569.05,N
4,DINHO,EDUARDO SUDRE,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,5357,60272747000120,AVULSO,60272747,14,KYO COMERCIAL AGRICOLA LTDA,SACOLAO SAUDE STO AN,J,ATIVO,0026,SACOLÃO SAÚDE,NORMAL,2008-09-01,2008-09-01,2022-12-16,11,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,9210580.0,AVENIDA DOS ESTADOS,PARQUE CENTRAL,SANTO ANDRÉ,2195,None,SANTO ANDRÉ,SP,11,49968888.0,8534.57,DINHO,RECEBIMENTO.SA@SACOLAOSAUDE.COM.BR,PESSOA JURIDICA - CONTRI NORMAL,ABC,-23.632460,-46.537459,NaN,6512.81,348.28,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,VIVIANCR,VIVIAN FIGUEIR,EQUIPE 2 (REP),ULISSES,ULISSES BACCI,11,942986864,ROGERIOF,ROGERIO FELIPIM,11,951283394,25283,41150000000155,AVULSO,41150000,14,MERCADO IRMAOS PEREIRA LIMA LTDA,MERCADO IRMAOS,J,ATIVO,0001,None,NORMAL,2021-03-25,NaT,2022-11-28,29,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,7131240.0,"RUA JOSEFINA DALCIN CASEIRO, 450",JARDIM DA MAMÃE,GUARULHOS,450,,GUARULHOS,SP,11,22796230.0,350.00,VIVIANCR,,PESSOA JURIDICA - CONTRI NORMAL,GUA,-23.434747,-46.522780,NaN,539.28,0.00,N
743,VIVIANCR,VIVIAN FIGUEIR,EQ